In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

Note: you may need to restart the kernel to use updated packages.


In [29]:
%pip install pandas
%pip install lxml

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 37.6 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
SCORE_DIR = "data/scores"

In [9]:
box_scores = os.listdir(SCORE_DIR)

In [10]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [13]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [19]:
import io

def read_line_score(soup):
    line_score = pd.read_html(io.StringIO(str(soup)), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]
    return line_score

In [33]:
def read_stats(soup, team, stat):
    df = pd.read_html(io.StringIO(str(soup)), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [53]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [54]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 8909
200 / 8909
300 / 8909
400 / 8909
500 / 8909
600 / 8909
700 / 8909
800 / 8909
900 / 8909
1000 / 8909
1100 / 8909
1200 / 8909
1300 / 8909
1400 / 8909
1500 / 8909
1600 / 8909
1700 / 8909
1800 / 8909
1900 / 8909
2000 / 8909
2100 / 8909
2200 / 8909
2300 / 8909
2400 / 8909
2500 / 8909
2600 / 8909
2700 / 8909
2800 / 8909
2900 / 8909
3000 / 8909
3100 / 8909
3200 / 8909
3300 / 8909
3400 / 8909
3500 / 8909
3600 / 8909
3700 / 8909
3800 / 8909
3900 / 8909
4000 / 8909
4100 / 8909
4200 / 8909
4300 / 8909
4400 / 8909
4500 / 8909
4600 / 8909
4700 / 8909
4800 / 8909
4900 / 8909
5000 / 8909
5100 / 8909
5200 / 8909
5300 / 8909
5400 / 8909
5500 / 8909
5600 / 8909
5700 / 8909
5800 / 8909
5900 / 8909
6000 / 8909
6100 / 8909
6200 / 8909
6300 / 8909
6400 / 8909
6500 / 8909
6600 / 8909
6700 / 8909
6800 / 8909
6900 / 8909
7000 / 8909
7100 / 8909
7200 / 8909
7300 / 8909
7400 / 8909
7500 / 8909
7600 / 8909
7700 / 8909
7800 / 8909
7900 / 8909
8000 / 8909
8100 / 8909
8200 / 8909
8300 / 8909
8400 / 8909
8

In [55]:
games_df = pd.concat(games, ignore_index=True)

In [58]:
games_df.to_csv("nba_games.csv")